In [8]:
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm

In [9]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

In [10]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['title_naver']
collection = db['naver_title_crawl']





def crawl(acode):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    pages = [1]
   
   
 
    search = "2021.08.19"
    seen_date = db.naver_title_crawl.find({"code":acode}).sort('date', -1).limit(1)
    
    #sort 전 어떤 변수에 저장해서, 
    #코드 기준으로 가져와서 데이트 정렬
    seen_date = list(seen_date)
    if not seen_date:
        
        seen_date = "2021.08.19 00:00"
    TF = False
    for page in pages:
        pages.append(page + 1)
        url = 'https://finance.naver.com/item/board.nhn?code={}&page={}'.format(acode, page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        table = dom.find('table', {'class' : 'type2'})
        tt = table.select('tbody > tr')

        #for k in tt[2:]:
         #   if len(k.select('td > span')) > 0:
               # date = k.select('td > span')[0].text
             
                #if str(date) <= str(seen_date):
                #    break
       # if search in str(date[:10]):
        #    break
       # else:
        for k in tt[2:]:
            if len(k.select('td > span')) > 0:
                date = k.select('td > span')[0].text
                if str(date) <= str(seen_date) or search == str(date[:10]):
                    TF = True
                    break
                title = k.select('td.title > a')[0]['title']
                views = k.select('td > span')[1].text
                pos = k.select('td > strong')[0].text
                neg = k.select('td > strong')[1].text


                put_data = {
                                 'code':acode,
                                 'date':date,
                                  'title':title,
                                  'views':views,
                                  'pos':pos,
                                   'neg':neg
                                             }

                db.naver_title_crawl.insert_one(put_data)
                
        if TF:
            break




In [11]:
acode={"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
      "ecopro":"086520","shinsungeng":"011930","unison":"018000",
      "kakao":"035720","naver":"035420","nc_s":"036570",
      "sds":"018260","samsung_e":"005930","sk_h":"000660",
      "skc":"011790","hansol":"014680","DB":"000990",
      "lotte_shoping":"023530","shinse":"004170","samsungcnt":"028260",
      "dongsuh":"026960","kb":"105560","korea":"071050","shinhan":"055550",
      "woori":"316140","hana":"086790"}

In [12]:
#10/3일까지 긁음. 추후 2주뒤에 10/3일 이후로 코드 수정하여 긁어와야 함.
#for code,v in acode.items():
#    result = crawl(v)


In [33]:
crawl("086790")